In [67]:
import math
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as numpy 

In [68]:
df= pd.read_csv("bank-additional-full.csv", sep= None, engine= "python")
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [69]:
d={'admin.':30,'management':24,'entrepreneur':20,'services':17,'self-employed':15,'technician':12,'blue-collar':10,'housemaid':6,
   'student':8,'retired':5,'unemployed':2,'unknown':1}
df['job']=df['job'].map(d)
d1={'success':30,'failure':7,'nonexistent':20}
df['poutcome']=df['poutcome'].map(d1)
df.loc[:,'duration'] *= 100
cols= ["age","poutcome","duration","campaign","pdays","previous","emp.var.rate","cons.price.idx","cons.conf.idx"]
data_encode= df.drop(cols, axis= 1)
data_encode= data_encode.apply(LabelEncoder().fit_transform)
data_rest= df[cols]
df= pd.concat([data_rest,data_encode], axis= 1)


In [70]:
data_train, data_test= train_test_split(df, test_size= 0.23, random_state= 1)
X_train= data_train.drop("y", axis= 1)
Y_train= data_train["y"]
X_test= data_test.drop("y", axis=1)
Y_test= data_test["y"]

In [71]:
scaler= StandardScaler()
scaler.fit(X_train)
X_train= scaler.transform(X_train)
X_test= scaler.transform(X_test)

In [72]:
K_cent= 8
km= KMeans(n_clusters= K_cent, max_iter= 100)
km.fit(X_train)
cent= km.cluster_centers_  

In [73]:
max=0 
for i in range(K_cent):
    for j in range(K_cent):
        d= numpy.linalg.norm(cent[i]-cent[j])
        if(d> max):
            max= d
sigma= max/math.sqrt(2*K_cent)


In [74]:
shape= X_train.shape
row= shape[0]
column= K_cent
G= numpy.empty((row,column), dtype= float)
for i in range(row):
    for j in range(column):
        dist= numpy.linalg.norm(X_train[i]-cent[j])
        G[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))

In [75]:
GTG= numpy.dot(G.T,G)
GTG_inv= numpy.linalg.inv(GTG)
fac= numpy.dot(GTG_inv,G.T)
W= numpy.dot(fac,Y_train)


In [76]:
row= X_test.shape[0]
column= K_cent
G_test= numpy.empty((row,column), dtype= float)
for i in range(row):
    for j in range(column):
        dist= numpy.linalg.norm(X_test[i]-cent[j])
        G_test[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))

In [77]:
prediction= numpy.dot(G_test,W)
prediction=numpy.round(prediction)
score= accuracy_score(prediction,Y_test)
print("The accuracy of whether the person will be getting loan or not is given by:- ")
print(score.mean())

The accuracy of whether the person will be getting loan or not is given by:- 
0.8943424108085286
